<h1><center>Image classification using a CNN</center></h1>

We first import the required libraries.

In [1]:
import os
import numpy as np
import requests, io, zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, ReLU

We then download and extract the data we will use.

In [2]:
if not os.path.exists('res'):
    train_req = requests.get('https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip')
    test_req = requests.get('https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip')
    files = zipfile.ZipFile(io.BytesIO(train_req.content))
    files.extractall('res')
    files = zipfile.ZipFile(io.BytesIO(test_req.content))
    files.extractall('res')

We will then create our datasets using the ImageDataGenerator tensorflow provides. For our train data we will also augment out data in order to further increase the model's accuracy.

In [3]:
train_dir = 'res/Training'
test_dir = 'res/Testing'

train_data_generator = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,   
    height_shift_range=0.1,   
    shear_range=0.1,          
    zoom_range=0.1,           
    horizontal_flip=True,     
    fill_mode='nearest',
    validation_split=0.2,
    rescale=1./255
)
validation_data_generator = ImageDataGenerator(validation_split=0.2, rescale=1./255)
test_data_generator = ImageDataGenerator(validation_split=0.2, rescale=1./255)

After creating the generators, we will create the datasets. Its useful to note that we will use the grayscale color mode and a very low resolution in order to read the images in a sufficient data frame.

In [11]:
train_dataset = train_data_generator.flow_from_directory(train_dir,
                                                        batch_size=32,
                                                        class_mode='categorical',
                                                        color_mode='grayscale',
                                                        target_size=(56,56),
                                                        subset='training')

validation_dataset = validation_data_generator.flow_from_directory(train_dir,
                                                        batch_size=32,
                                                        class_mode='categorical',
                                                        color_mode='grayscale',
                                                        target_size=(56,56),
                                                        subset='validation')

test_dataset =  test_data_generator.flow_from_directory(test_dir,
                                                  batch_size=32,
                                                  class_mode  = 'categorical',
                                                  color_mode='grayscale',
                                                  target_size=(56,56))

Found 3683 images belonging to 62 classes.
Found 892 images belonging to 62 classes.
Found 2520 images belonging to 62 classes.


Now, we will create and compile our models. We will start with the custom one.

In [25]:
model = tf.keras.models.Sequential()
model.add(Conv2D(16, (5, 5), strides=(1, 1), activation=None, padding='same', input_shape=(56, 56, 1)))
model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Dropout(0.1))

model.add(Conv2D(32, (5, 5), activation=None, padding='same'))
model.add(Conv2D(16, (5, 5), activation=None, padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Dropout(0.1))

model.add(Conv2D(16, (5, 5), activation=None, padding='same'))
model.add(Conv2D(8, (5, 5), activation=None, padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(62, activation='softmax'))
model.compile(optimizer= tf.keras.optimizers.Adam(),
            loss = 'categorical_crossentropy',
            metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_38 (Conv2D)          (None, 56, 56, 16)        416       
                                                                 
 conv2d_39 (Conv2D)          (None, 56, 56, 32)        12832     
                                                                 
 batch_normalization_18 (Bat  (None, 56, 56, 32)       128       
 chNormalization)                                                
                                                                 
 re_lu (ReLU)                (None, 56, 56, 32)        0         
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 28, 28, 32)       0         
 g2D)                                                            
                                                                 
 dropout_22 (Dropout)        (None, 28, 28, 32)       

It is time to train the model and check the results.

In [26]:
callbacks = []

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'custom_model_weights.hdf5', save_best_only=True, verbose=0)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=25, restore_best_weights=True, verbose=0)
callbacks.append(early_stop_callback)

history = model.fit(train_dataset,
                              validation_data=validation_dataset,
                              steps_per_epoch=len(train_dataset),
                              epochs=100,
                              validation_steps=len(validation_dataset),
                              verbose=1,
                              callbacks=callbacks)

Epoch 1/100
116/116 [==============================] - 4s 25ms/step - loss: 2.9163 - accuracy: 0.3065 - val_loss: 2.3403 - val_accuracy: 0.4238
Epoch 2/100
116/116 [==============================] - 3s 22ms/step - loss: 1.7753 - accuracy: 0.5300 - val_loss: 1.8621 - val_accuracy: 0.4832
Epoch 3/100
116/116 [==============================] - 3s 22ms/step - loss: 1.2889 - accuracy: 0.6364 - val_loss: 1.0374 - val_accuracy: 0.7354
Epoch 4/100
116/116 [==============================] - 3s 23ms/step - loss: 1.0583 - accuracy: 0.6967 - val_loss: 0.6108 - val_accuracy: 0.8094
Epoch 5/100
116/116 [==============================] - 3s 22ms/step - loss: 0.9193 - accuracy: 0.7206 - val_loss: 0.6595 - val_accuracy: 0.8038
Epoch 6/100
116/116 [==============================] - 3s 22ms/step - loss: 0.7887 - accuracy: 0.7559 - val_loss: 0.5105 - val_accuracy: 0.8408
Epoch 7/100
116/116 [==============================] - 3s 22ms/step - loss: 0.6881 - accuracy: 0.7793 - val_loss: 0.6551 - val_accuracy:

In [27]:
loss, acc = model.evaluate(test_dataset)

79/79 [==============================] - 15s 192ms/step - loss: 0.2323 - accuracy: 0.9429


As we can see we have a very accurate model for our dataset. Also, the test accuracy is on part with the training accuracy, suggesting we have no overfitting or underfitting. How will this model perform if we use it in conjuction with a pretrained model tho? We will use the MobileNetV2 pretrained model as we use quite low resolution images.

In [4]:
import gc
from keras.applications import MobileNetV2
gc.collect()

train_dataset_rgb = train_data_generator.flow_from_directory(train_dir,
                                                        batch_size=32,
                                                        class_mode='categorical',
                                                        color_mode='rgb',
                                                        target_size=(56,56),
                                                        subset='training')

validation_dataset_rgb = validation_data_generator.flow_from_directory(train_dir,
                                                        batch_size=32,
                                                        class_mode='categorical',
                                                        color_mode='rgb',
                                                        target_size=(56,56),
                                                        subset='validation')

test_dataset_rgb =  test_data_generator.flow_from_directory(test_dir,
                                                  batch_size=32,
                                                  class_mode  = 'categorical',
                                                  color_mode='rgb',
                                                  target_size=(56,56))

Found 3683 images belonging to 62 classes.
Found 892 images belonging to 62 classes.
Found 2520 images belonging to 62 classes.


For the pretrained model we want to test how our custom model will work as the top part of our pretrained weights. For that reason we will freeze all base weights except a few. As the pretrained weights might not be trained for our specific dataset, we will expect a slower accuracy increase through the epochs, and potentially a lower final accuracy score.

In [9]:
pretrained_base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(56, 56, 3))
#pretrained_base_model.trainable = False
pretrained_base_model.trainable = True
for layer in pretrained_base_model.layers[:-5]:
    layer.trainable = False
input_layer = tf.keras.layers.Input(shape=(56, 56, 3))
x = pretrained_base_model(input_layer)
x = Conv2D(16, (5, 5), strides=(1, 1), activation=None, padding='same', input_shape=(56, 56, 3))(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.1)(x)

#Custom Convolutional Layer 2
x = Conv2D(32, (5, 5), activation=None, padding='same')(x)
x = Conv2D(16, (5, 5), activation=None, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.1)(x)

# Custom Convolutional Layer 3
x = Conv2D(16, (5, 5), activation=None, padding='same')(x)
x = Conv2D(8, (5, 5), activation=None, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.1)(x)

# Step 6: Flatten and add Dense layers for final classification
x = Flatten()(x)
x = Dense(200, activation='relu')(x)
x = Dropout(0.25)(x)
output = Dense(62, activation='softmax')(x)

# Step 7: Create the complete model
model = Model(inputs=input_layer, outputs=output)

# Step 8: Compile the model
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 9: Print the model summary
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 56, 56, 3)]       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 2, 2, 1280)       2257984   
 ional)                                                          
                                                                 
 conv2d_3 (Conv2D)           (None, 2, 2, 16)          512016    
                                                                 
 batch_normalization_2 (Batc  (None, 2, 2, 16)         64        
 hNormalization)                                                 
                                                                 
 re_lu_2 (ReLU)              (None, 2, 2, 16)          0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 1, 16)         0   

The only difference between the process of the model creations is the learning rate. We use SGD instead of Adam. The reason is because of the greater amount of trainable weights as well as the existence of non trainable weights, the model may need to adjust the learning rate dynamically.

In [10]:
callbacks_pretrained = []

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'pretrainde_model_weights.hdf5', save_best_only=True, verbose=0)
callbacks_pretrained.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=25, restore_best_weights=True, verbose=0)
callbacks_pretrained.append(early_stop_callback)

history_pretrained = model.fit(train_dataset_rgb,
                              validation_data=validation_dataset_rgb,
                              steps_per_epoch=len(train_dataset_rgb),
                              epochs=100,
                              validation_steps=len(validation_dataset_rgb),
                              verbose=1,
                              callbacks=callbacks_pretrained)

Epoch 1/100
116/116 [==============================] - 8s 48ms/step - loss: 3.4768 - accuracy: 0.2034 - val_loss: 3.5446 - val_accuracy: 0.1928
Epoch 2/100
116/116 [==============================] - 5s 40ms/step - loss: 2.7022 - accuracy: 0.2930 - val_loss: 2.7312 - val_accuracy: 0.3128
Epoch 3/100
116/116 [==============================] - 5s 42ms/step - loss: 2.4015 - accuracy: 0.3739 - val_loss: 2.6066 - val_accuracy: 0.3587
Epoch 4/100
116/116 [==============================] - 4s 38ms/step - loss: 2.2283 - accuracy: 0.4021 - val_loss: 2.7121 - val_accuracy: 0.3576
Epoch 5/100
116/116 [==============================] - 5s 41ms/step - loss: 2.1054 - accuracy: 0.4295 - val_loss: 2.3946 - val_accuracy: 0.3733
Epoch 6/100
116/116 [==============================] - 6s 49ms/step - loss: 2.0018 - accuracy: 0.4363 - val_loss: 2.3823 - val_accuracy: 0.4316
Epoch 7/100
116/116 [==============================] - 5s 40ms/step - loss: 1.9348 - accuracy: 0.4602 - val_loss: 2.5299 - val_accuracy:

We can see that the accuracy stops improving at around 61%. Then we witness overfitting. That suggests the algorithm could be optimized better for use with a pretrained model.

In [11]:
loss, acc = model.evaluate(test_dataset_rgb)

79/79 [==============================] - 2s 21ms/step - loss: 1.0845 - accuracy: 0.6560


The accuracy is slightly higher than than of our training dataset. Despite the quite lower accuracy, ~65% is quite a commendable amount considering half our weights are not trainable and the pretrained dataset propably is not comparable to our data.